In [3]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/chisel-bootcamp/source/load-ivy.sc"

In [4]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import dotvisualizer._

In [5]:
class DES extends Module {
  val io = IO(new Bundle {
    val plaintext  = Input(UInt(64.W))
    val key        = Input(UInt(64.W))
    val start      = Input(Bool())
    val done       = Output(Bool())
    val ciphertext = Output(UInt(64.W))
  })

  private def permuteN(inBits: UInt, table: Seq[Int], inWidth: Int): UInt = {
    Cat(table.map(i => inBits(inWidth - i)).toSeq)
  }

  private val IP = Seq(58,50,42,34,26,18,10,2, 60,52,44,36,28,20,12,4, 62,54,46,38,30,22,14,6, 64,56,48,40,32,24,16,8, 57,49,41,33,25,17,9,1, 59,51,43,35,27,19,11,3, 61,53,45,37,29,21,13,5, 63,55,47,39,31,23,15,7)
  private val FP = Seq(40,8,48,16,56,24,64,32, 39,7,47,15,55,23,63,31, 38,6,46,14,54,22,62,30, 37,5,45,13,53,21,61,29, 36,4,44,12,52,20,60,28, 35,3,43,11,51,19,59,27, 34,2,42,10,50,18,58,26, 33,1,41,9,49,17,57,25)
  private val E  = Seq(32,1,2,3,4,5, 4,5,6,7,8,9, 8,9,10,11,12,13, 12,13,14,15,16,17, 16,17,18,19,20,21, 20,21,22,23,24,25, 24,25,26,27,28,29, 28,29,30,31,32,1)
  private val P  = Seq(16,7,20,21,29,12,28,17, 1,15,23,26,5,18,31,10, 2,8,24,14,32,27,3,9, 19,13,30,6,22,11,4,25)
  private val PC1= Seq(57,49,41,33,25,17,9, 1,58,50,42,34,26,18, 10,2,59,51,43,35,27, 19,11,3,60,52,44,36, 63,55,47,39,31,23,15, 7,62,54,46,38,30,22, 14,6,61,53,45,37,29, 21,13,5,28,20,12,4)
  private val PC2= Seq(14,17,11,24,1,5,3,28, 15,6,21,10,23,19,12,4, 26,8,16,7,27,20,13,2, 41,52,31,37,47,55,30,40, 51,45,33,48,44,49,39,56, 34,53,46,42,50,36,29,32)
  private val SH  = Seq(1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1)

  private val sboxRaw: Seq[Seq[Seq[Int]]] = Seq(
    Seq(Seq(14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7), Seq(0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8), Seq(4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0), Seq(15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13)),
    Seq(Seq(15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10), Seq(3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5), Seq(0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15), Seq(13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9)),
    Seq(Seq(10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8), Seq(13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1), Seq(13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7), Seq(1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12)),
    Seq(Seq(7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15), Seq(13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9), Seq(10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4), Seq(3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14)),
    Seq(Seq(2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9), Seq(14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6), Seq(4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14), Seq(11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3)),
    Seq(Seq(12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11), Seq(10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8), Seq(9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6), Seq(4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13)),
    Seq(Seq(4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1), Seq(13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6), Seq(1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2), Seq(6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12)),
    Seq(Seq(13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7), Seq(1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2), Seq(7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8), Seq(2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11))
  )
  private val sboxes = VecInit(sboxRaw.map { rows =>
    VecInit(rows.map(row => VecInit(row.map(_.U(4.W)))))
  })

private def sboxLayer(x: UInt): UInt = {
  require(x.getWidth == 48)
  val outs = (0 until 8).map { i =>
    val hi = 47 - i*6
    val lo = hi - 5
    val chunk = x(hi, lo)   
    val row = Cat(chunk(5), chunk(0))
    val col = chunk(4,1)
    sboxes(i)(row)(col) 
  }
  outs.reduce(Cat(_, _))   
}

  private def subkeys56(k64: UInt): Vec[UInt] = {
    val k56 = permuteN(k64, PC1, 64) // drop parity
    val c0 = k56(55,28)
    val d0 = k56(27,0)

    def rol28(x: UInt, n: Int): UInt = ((x << n)(27,0)) | (x >> (28 - n))

    val c = Wire(Vec(17, UInt(28.W)))
    val d = Wire(Vec(17, UInt(28.W)))
    c(0) := c0; d(0) := d0
    for (i <- 0 until 16) {
      c(i+1) := rol28(c(i), SH(i))
      d(i+1) := rol28(d(i), SH(i))
    }
    val ks = Wire(Vec(16, UInt(48.W)))
    for (i <- 0 until 16) {
      ks(i) := permuteN(Cat(c(i+1), d(i+1)), PC2, 56)
    }
    ks
  }

  private def fFunc(r: UInt, k: UInt): UInt = {
    val e = permuteN(r, E, 32)          // 32 -> 48
    val x = e ^ k
    val s = sboxLayer(x)                // 48 -> 32
    permuteN(s, P, 32)
  }

  //FSM
  val idle :: busy :: done :: Nil = Enum(3)
  val st = RegInit(idle)
  val round = RegInit(0.U(5.W))
  val L = Reg(UInt(32.W))
  val R = Reg(UInt(32.W))
  val K = subkeys56(io.key)

  io.done := false.B
  io.ciphertext := 0.U

  switch(st) {
    is(idle) {
      when(io.start) {
        val ip = permuteN(io.plaintext, IP, 64)
        L := ip(63,32); R := ip(31,0)
        round := 0.U
        st := busy
      }
    }
    is(busy) {
      val f = fFunc(R, K(round))
      val newL = R
      val newR = L ^ f
      L := newL; R := newR
      when(round === 15.U) {
        val preOut = Cat(newR, newL)
        io.ciphertext := permuteN(preOut, FP, 64)
        io.done := true.B
        st := done
      }.otherwise {
        round := round + 1.U
      }
    }
    is(done) {
      when(!io.start) { st := idle }
    }
  }
}

defined class DES

In [6]:
{
  test(new DES) { dut =>
    val pt  = BigInt("123456ABCD132536", 16)
    val key = BigInt("AABB09182736CCDD", 16)
    val exp = BigInt("C0B7A8D05F3A829C", 16)

    dut.io.plaintext.poke(pt.U)
    dut.io.key.poke(key.U)
    dut.io.start.poke(true.B)
    dut.clock.step(1)
    dut.io.start.poke(false.B)
    while (!dut.io.done.peek().litToBoolean) { dut.clock.step(1) }

    val out = dut.io.ciphertext.peek().litValue
    println(f"Ciphertext: 0x$out%016x")
    assert(out == exp)
  }
}

Elaborating design...
Done elaborating.
Ciphertext: 0xc0b7a8d05f3a829c
test DES Success: 0 tests passed in 18 cycles in 0.172616 seconds 104.28 Hz
